In [ ]:

!cp /content/drive/MyDrive/handmask/hands.pth.tar -r /content/weights


In [ ]:
!git clone https://github.com/adarsh1001/Hand_Segmentation_RefineNet
!cp /content/Hand_Segmentation_RefineNet/RefineNet/refinenet.py -r /content
!cp /content/Hand_Segmentation_RefineNet/RefineNet/weights -r /content
!cp /content/drive/MyDrive/handmask/hands.pth.tar -r /content/weights



Cloning into 'Hand_Segmentation_RefineNet'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.


In [ ]:
import os
import glob
from pathlib import Path
from csv import writer
import cv2
from cv2 import VideoCapture, waitKey, imshow, cvtColor, flip, COLOR_BGR2RGB, COLOR_RGB2BGR, putText, FONT_HERSHEY_SIMPLEX, resize, rectangle, line, VideoWriter_fourcc, VideoWriter
from numpy import asscalar, ndarray, array, zeros, concatenate, load, uint8, set_printoptions, inf
from pathlib import Path
from shutil import rmtree
import random
from os.path import split
import matplotlib.pyplot as plt
import re
import torch
from refinenet import refinenet
from tqdm import tqdm

cmap = np.load('weights/cmap.npy')
img_dir = '/content/images/'


# vid_dir = '/content/video'
# vids = os.listdir(vid_dir)

numC = 7

img_scale = 1./255
img_mean = np.array([0.485, 0.456, 0.406]).reshape((1, 1, 3))
img_std = np.array([0.229, 0.224, 0.225]).reshape((1, 1, 3))

net = refinenet(numC, pretrained=True).eval()
net = net.cuda() #Run only with Cuda support!
idx = 1


current_ds = '/content/Age_2.avi'
video = current_ds
video_cap = VideoCapture(str(video))
flag, frame = video_cap.read()
cnt =1
frame_length = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))
rmtree('/content/images')
Path('/content/images').mkdir()

while flag:
    cv2.imwrite('/content/images/frame%d.jpg' %cnt, frame)
    flag, frame = video_cap.read()
    cnt+=1


imgs = os.listdir(img_dir)
if Path('/content/segmented_images/').exists():rmtree('/content/segmented_images/')
Path('/content/segmented_images/').mkdir()
with torch.no_grad():

    for img in imgs:
        if img!='.ipynb_checkpoints':
            img_path = img_dir + img
            # img_path = '/content/new_images/hand.webp'
            print(img)
            img_name=img
            img = np.array(plt.imread(img_path))
            
            orig_size = img.shape[:2][::-1]

            inter_img = (img*img_scale - img_mean)/img_std

            img_inp = torch.tensor(inter_img.transpose(2, 0, 1)[None]).float()
            img_inp = img_inp.cuda()

            seg = net(img_inp)[0].data.cpu().numpy().transpose(1, 2, 0)
            seg = cv2.resize(seg, orig_size, interpolation=cv2.INTER_CUBIC)
            seg = cmap[seg.argmax(axis=2).astype(np.uint8)]

            ma = (seg[:,:,0]>=0) & (seg[:,:,0]<=20) & (seg[:,:,2]>=110) & (seg[:,:,2]<=150) & (seg[:,:,1]>=0) & (seg[:,:,1]<=20)
            x = np.zeros(ma.shape)
            x[ma==True] = 255
            x[ma==False] = 0
            print(len(x))
            import numpy
            # with numpy.printoptions(threshold=numpy.inf):
            #     print(x)
            #     break
            print('/content/segmented_images/'+str(img_name))
            cv2.imwrite('/content/segmented_images/'+str(img_name),x)
            idx += 1
            print('done')


In [ ]:
segmented_images_folder = ('/content/segmented_images')
video_name = ('/content/sample.avi')
images = [img for img in os.listdir(segmented_images_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(segmented_images_folder, images[0]))
height, width, layers = frame.shape
print(images)
video = cv2.VideoWriter(video_name, 0, 10, (width,height))

for img in images:
    frame = np.fromfile('segmented_images/'+img, dtype='int16', sep="")
    print(frame)
    video.write(frame)

['frame2.jpg', 'frame39.jpg', 'frame35.jpg', 'frame42.jpg', 'frame30.jpg', 'frame41.jpg', 'frame10.jpg', 'frame43.jpg', 'frame15.jpg', 'frame46.jpg', 'frame7.jpg', 'frame34.jpg', 'frame11.jpg', 'frame16.jpg', 'frame12.jpg', 'frame32.jpg', 'frame47.jpg', 'frame24.jpg', 'frame37.jpg', 'frame29.jpg', 'frame50.jpg', 'frame31.jpg', 'frame33.jpg', 'frame5.jpg', 'frame22.jpg', 'frame17.jpg', 'frame8.jpg', 'frame19.jpg', 'frame28.jpg', 'frame40.jpg', 'frame45.jpg', 'frame21.jpg', 'frame3.jpg', 'frame44.jpg', 'frame26.jpg', 'frame14.jpg', 'frame18.jpg', 'frame1.jpg', 'frame23.jpg', 'frame38.jpg', 'frame6.jpg', 'frame25.jpg', 'frame13.jpg', 'frame27.jpg', 'frame9.jpg', 'frame48.jpg', 'frame49.jpg', 'frame20.jpg', 'frame4.jpg', 'frame36.jpg']
[-9985 -7937  4096 ...  5189 24401 -9729]


error: ignored

In [ ]:
import cv2
import numpy as np
from pathlib import Path

all_img_path = list((Path("/content/segmented_images")).glob("*"))
print(len(all_img_path))
# choose codec according to format needed


50


In [ ]:
!zip -r /content/sample.zip /content/segmented_images